# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

### Import With: mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

In [136]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [137]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [138]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'fruits_db', 'local', 'met', 'uk_food']


In [139]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [140]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [141]:
# review the collections in our new database
pprint(db.find)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food'), 'find')


In [142]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6468eaf93a8ed94777e77774'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [143]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [144]:
# Create a dictionary for the new restaurant data
halal_data = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}


In [145]:
# Insert the new restaurant into the collection
establishments.insert_one(halal_data)

In [146]:
# Check that the new restaurant was inserted
new_restaurant = {"BusinessName" : "Penang Flavours"}
pprint(establishments.find_one(new_restaurant))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6468eb01702cf99de376866f'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


In [147]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType" : "Restaurant/Cafe/Canteen"}
filter = {'BusinessType' : 1, 'BusinessTypeID' : 1}
document = establishments.find_one(query, filter)
pprint(document)

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6468eaf93a8ed94777e77774')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [148]:
# Update the new restaurant with the correct BusinessTypeID
query = {"BusinessName" : new_restaurant['BusinessName']}
new_values = { "$set": { "BusinessTypeID" : document['BusinessTypeID'] }}
establishments.update_one(query, new_values)

In [149]:
# Confirm that the new restaurant was updated
pprint(establishments.find_one({"BusinessName" : "Penang Flavours"}, {"BusinessName" : 1, "BusinessTypeID" : 1}))

{'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1,
 '_id': ObjectId('6468eb01702cf99de376866f')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [150]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName" : "Dover"}
result = establishments.find(query)
result = list(result)
print(f"Dover has {len(result)} establishments")

Dover has 994 establishments


In [151]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {"LocalAuthorityName" : "Dover"}
establishments.delete_many(query)

In [152]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName" : "Dover"}
len(list(establishments.find(query)))

0

In [153]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6468eaf93a8ed94777e77a5b'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [154]:
# Change the data type from String to Decimal for longitude and latitude
conversion = {
            '$set' : {
                    'geocode.latitude' : { 
                        '$toDecimal' : '$geocode.latitude' 
                        },
                    'geocode.longitude' : { 
                        '$toDecimal' : '$geocode.longitude' 
                    }
                }
        }
filter = {}
establishments.update_many(filter, [conversion])

Use `update_many` to convert `RatingValue` to integer numbers.

In [155]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [156]:
# Change the data type from String to Integer for RatingValue
conversion = {
            '$set' : {
                    'RatingValue' : { 
                        '$toInt' : '$RatingValue' 
                        }
                }
        }
filter = {}
establishments.update_many(filter, [conversion])

In [157]:
# Check that the coordinates and rating value are now numbers
query = {}
filter = {'RatingValue' : 1, "geocode" : 1}
pprint(establishments.find_one(query, filter))

{'RatingValue': 5,
 '_id': ObjectId('6468eaf93a8ed94777e77a5b'),
 'geocode': {'latitude': Decimal128('51.083812'),
             'longitude': Decimal128('1.195625')}}
